Dataset “Babyboom” (переменные Time of birth recorded on the 24-hour clock, Sex of the child (1 = girl, 2 = boy), Birth weight in grams, Number of minutes after midnight of each birth): 

- Проверьте вес младенцев на нормальность. Сначала используйте все данные, не разделяя детей по полу, затем для группы мальчиков и девочек по отдельности. При проверке гипотез используйте точечные оценки параметров. Постройте доверительные интервалы для параметров нормального распределения. 

- Проверьте гипотезу о том, что время между рождением детей подчиняется экспоненциальному распределению. При проверке гипотезы используйте точечные оценки параметров. 

- Проверьте гипотезу, подчиняется ли количество рождений в час для каждого часа распределению Пуассона. При проверке гипотезы используйте точечные оценки параметров. 

Dataset “Euroweight” (переменные weight, batch): 

- Проверить гипотезу о нормальном распределении веса монет (сначала объединить все монеты в одну выборку, потом по каждому пакету в отдельности). При проверке гипотез используйте точечные оценки параметров.  

- Постройте доверительные интервалы для параметров нормального распределения. 

Dataset “iris.txt” (прочитайте описание данных в файле «iris_description.txt», переменные sepal length, sepal width, petal length, petal width, class): 

- Проверить гипотезу о нормальном распределении длины цветков, сгруппировав их по типу ириса. При проверке гипотез используйте точечные оценки параметров.  

- Постройте доверительные интервалы для параметров нормального распределения. 

# Babyboom

In [1]:
import numpy as np
import pandas as pd

from scipy import stats

In [2]:
# Определяем позиции столбцов
column_specs = [
    (0, 8),   # TimeOfBirth
    (8, 16),  # Sex
    (16, 24),  # BirthWeight
    (24, 32),  # NumOfMinutesAfterMidnight
]

# Определяем имена столбцов
column_names = [
    "TimeOfBirth",
    "Sex",
    "BirthWeight",
    "NumOfMinutesAfterMidnight",
]

# Читаем файл
df = pd.read_fwf('babyboomdat.txt', colspecs=column_specs, header=None, names=column_names)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   TimeOfBirth                44 non-null     int64
 1   Sex                        44 non-null     int64
 2   BirthWeight                44 non-null     int64
 3   NumOfMinutesAfterMidnight  44 non-null     int64
dtypes: int64(4)
memory usage: 1.5 KB


In [4]:
data = df['BirthWeight']

statistic, p_value = stats.shapiro(data)

print(f'ALL: W={statistic}, p-value={p_value}')

data_boys = df[df['Sex'] == 2]['BirthWeight']

statistic, p_value = stats.shapiro(data_boys)

print(f'BOYS: W={statistic}, p-value={p_value}')

data_girls = df[df['Sex'] == 1]['BirthWeight']

statistic, p_value = stats.shapiro(data_girls)

print(f'GIRLS: W={statistic}, p-value={p_value}')

ALL: W=0.8987229755100873, p-value=0.0009943974027275867
BOYS: W=0.9474738942208476, p-value=0.20224825440795427
GIRLS: W=0.8702829450561838, p-value=0.017984789994719325


In [5]:
def get_conf_interval(data, confidence=0.95):
    mean = np.mean(data)
    std_err = stats.sem(data)
    h = std_err * stats.t.ppf((1 + confidence) / 2, df=len(data) - 1)

    return mean, mean - h, mean + h

mean, low, high = get_conf_interval(data)
print(f'ALL: MEAN={mean}, Interval=({low}, {high})')
mean, low, high = get_conf_interval(data_boys)
print(f'BOYS: MEAN={mean}, Interval=({low}, {high})')
mean, low, high = get_conf_interval(data_girls)
print(f'GIRLS: MEAN={mean}, Interval=({low}, {high})')

ALL: MEAN=3275.9545454545455, Interval=(3115.418005028038, 3436.491085881053)
BOYS: MEAN=3375.3076923076924, Interval=(3202.4161508132743, 3548.1992338021105)
GIRLS: MEAN=3132.4444444444443, Interval=(2818.3658395570424, 3446.5230493318463)


In [7]:
time_between_births = df['NumOfMinutesAfterMidnight'].diff().dropna()

l = 1 / time_between_births.mean()
d, p_value = stats.kstest(time_between_births, 'expon', args=(0, 1/l))

print(f'EXP: D={d}, p-value={p_value}')

EXP: D=0.13751184043266534, p-value=0.3574267136129613


In [8]:
df['Hour'] = df['NumOfMinutesAfterMidnight'] // 60
births_per_hour = df['Hour'].value_counts().sort_index()

l = births_per_hour.mean()

d, p_value = stats.kstest(births_per_hour, 'poisson', args=(l,))
print(f'POISSON: D={d}, p-value={p_value}')

POISSON: D=0.3808410193191538, p-value=0.003010055985884885


In [27]:
from scipy.stats import poisson, chisquare

sample_size = len(df)

df['Hour'] = df['NumOfMinutesAfterMidnight'] // 60
births_per_hour, bins = np.histogram(df['Hour'], bins=np.arange(25))
lambda_poisson = np.mean(births_per_hour)
expected_counts = poisson.pmf(np.arange(24), mu=lambda_poisson) * sample_size

chi_stat, p_value = chisquare(births_per_hour, expected_counts)

print(f'POISSON: Statistic={chi_stat}, p-value={p_value}')

POISSON: Statistic=1.3328777274900886e+16, p-value=0.0


# Euroweight

In [11]:
# Определяем имена столбцов
column_names = [
    "id",
    "weight",
    "batch",
]

# Читаем файл
df = pd.read_csv('euroweightdat.txt', sep='\t', header=None, names=column_names)

In [12]:
df

,id,weight,batch
0,1,7.512,1
1,2,7.502,1
2,3,7.461,1
3,4,7.562,1
4,5,7.528,1
...,...,...,...
1995,1996,7.514,8
1996,1997,7.519,8
1997,1998,7.606,8
1998,1999,7.547,8


In [19]:
data = df['weight']

ks_stat, p_value = stats.kstest(data, 'norm', args=(data.mean(), data.std()))

print(f'ALL: KS Statistic={ks_stat}, p-value={p_value}')

for batch in df['batch'].unique():
    data = df[df['batch'] == batch]['weight']
    ks_stat, p_value = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f'Batch N{batch}: KS Statistic={ks_stat}, p-value={p_value}')

ALL: KS Statistic=0.023353803446905852, p-value=0.22192512324841585
Batch N1: KS Statistic=0.038472438278823295, p-value=0.8389620125716332
Batch N2: KS Statistic=0.03265930919143112, p-value=0.9444808004928708
Batch N3: KS Statistic=0.07792267647627489, p-value=0.0910506298988909
Batch N4: KS Statistic=0.04544943866051099, p-value=0.6628215726419289
Batch N5: KS Statistic=0.03499095797591667, p-value=0.9087939122045151
Batch N6: KS Statistic=0.055099573639602606, p-value=0.4186860218860704
Batch N7: KS Statistic=0.042332335662710996, p-value=0.7449928480338537
Batch N8: KS Statistic=0.0701373917985999, p-value=0.16294891063512817


In [21]:
data = df['weight']

mean, low, high = get_conf_interval(data)

print(f'ALL: MEAN={mean}, Interval=({low}, {high})')

for batch in df['batch'].unique():
    data = df[df['batch'] == batch]['weight']
    mean, low, high = get_conf_interval(data)
    print(f'Batch N{batch}: MEAN={mean}, Interval=({low}, {high})')

ALL: MEAN=7.5212325, Interval=(7.519724882523983, 7.522740117476017)
Batch N1: MEAN=7.519656, Interval=(7.515375800837306, 7.523936199162694)
Batch N2: MEAN=7.523168000000001, Interval=(7.518747761295764, 7.527588238704238)
Batch N3: MEAN=7.509544, Interval=(7.504930027624747, 7.514157972375253)
Batch N4: MEAN=7.531104, Interval=(7.5274397498218875, 7.534768250178113)
Batch N5: MEAN=7.531396000000001, Interval=(7.527705750509117, 7.535086249490885)
Batch N6: MEAN=7.5152399999999995, Interval=(7.511076559213748, 7.519403440786251)
Batch N7: MEAN=7.523015999999999, Interval=(7.518905563172776, 7.527126436827222)
Batch N8: MEAN=7.516736, Interval=(7.512205019334981, 7.521266980665018)


# Iris

In [22]:
# Определяем имена столбцов
column_names = [
    "sepal_length",
    "sepal_width",
    "petal_length",
    "petal_width",
    "class",
]

# Читаем файл
df = pd.read_csv('iris.txt', header=None, names=column_names)

In [23]:
df

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [25]:
for fclass in df['class'].unique():
    data = df[df['class'] == fclass]['sepal_length']
    ks_stat, p_value = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
    print(f'Class {fclass}: KS Statistic={ks_stat}, p-value={p_value}')

Class Iris-setosa: KS Statistic=0.11485990669608126, p-value=0.4889236515009082
Class Iris-versicolor: KS Statistic=0.09624091191683509, p-value=0.7071234035717402
Class Iris-virginica: KS Statistic=0.11503428728233644, p-value=0.48699833606145904


In [26]:
for fclass in df['class'].unique():
    data = df[df['class'] == fclass]['sepal_length']
    mean, low, high = get_conf_interval(data)
    print(f'Class {fclass}: MEAN={mean}, Interval=({low}, {high})')

Class Iris-setosa: MEAN=5.006, Interval=(4.905823539299264, 5.106176460700737)
Class Iris-versicolor: MEAN=5.936, Interval=(5.789305783106826, 6.082694216893174)
Class Iris-virginica: MEAN=6.587999999999998, Interval=(6.407285019117501, 6.768714980882495)
